Ejecutado desde VERTEX AI Notebooks Colab Enterprise

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import json
import os
import tempfile
from google.cloud import storage

### Business

Configuración de Paths de entrada y salida



In [2]:
bucket_name = "json-bucket-datos"
input_folder_name = "Yelp"
input_file_name = "business.pkl"
output_folder_name = "Yelp/ETL"
output_file_name = "business.parquet"

# Crear cliente de Google Cloud Storage
storage_client = storage.Client()

# Construir la referencia al archivo de origen en la carpeta especificada
bucket = storage_client.get_bucket(bucket_name)
blob_name = os.path.join(input_folder_name, input_file_name)
input_blob = bucket.blob(blob_name)

# Crear directorio temporal para trabajar con los archivos
temp_dir = tempfile.mkdtemp()

# Asignar el archivo de origen al directorio temporal
input_file_path = os.path.join(temp_dir, input_file_name)
input_blob.download_to_filename(input_file_path)

# Crear referencia al archivo de salida de Parquet en la misma carpeta
output_blob_name = os.path.join(output_folder_name, output_file_name)
output_blob = bucket.blob(output_blob_name)

# Vamos a guardar los datos transformados en un nuevo archivo de Parquet
output_file_path = os.path.join(temp_dir, output_file_name)

In [3]:
# Leer pickle a pandas
df = pd.read_pickle(input_file_path)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

In [5]:
# Eliminar duplicados
df = df.loc[:, ~df.columns.duplicated()]

# Desplazar la columna hacia arriba
df["state"] = df["state"].shift(-3)

<ipython-input-5-f320263e72cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["state"] = df["state"].shift(-3)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


Filtramos por Hoteles Holiday Inn y eliminamos registros con datos inadecuados (verificados manual en EDA)

In [6]:
# Filtro que aisla hoteles Holiday Inn
filtro = (df['name'].str.contains(r'\bHoliday Inn'))

df_holiday = df.loc[filtro]

# Eliminar la primera fila
df_holiday = df_holiday.drop(df_holiday.index[0])

# Eliminar la última fila
df_holiday = df_holiday.drop(df_holiday.index[-1])

In [7]:
df_holiday.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 4006 to 148306
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   144 non-null    object
 1   name          144 non-null    object
 2   address       144 non-null    object
 3   city          144 non-null    object
 4   state         144 non-null    object
 5   postal_code   144 non-null    object
 6   latitude      144 non-null    object
 7   longitude     144 non-null    object
 8   stars         144 non-null    object
 9   review_count  144 non-null    object
 10  is_open       144 non-null    object
 11  attributes    141 non-null    object
 12  categories    144 non-null    object
 13  hours         126 non-null    object
dtypes: object(14)
memory usage: 16.9+ KB


Filtramos sólo hoteles de la competencia

In [8]:
# Lista Competidores
lista_competidores = ["best western", "hyatt", "radisson", "marriott", "hilton", "wyndham", "belmond", "accor", "Meliá"]

# Filtrar el DataFrame con las condiciones
df_competidores = df[df['name'].str.lower().str.contains('|'.join(lista_competidores))]

In [11]:
df_competidores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 482 to 150331
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   757 non-null    object
 1   name          757 non-null    object
 2   address       757 non-null    object
 3   city          757 non-null    object
 4   state         757 non-null    object
 5   postal_code   757 non-null    object
 6   latitude      757 non-null    object
 7   longitude     757 non-null    object
 8   stars         757 non-null    object
 9   review_count  757 non-null    object
 10  is_open       757 non-null    object
 11  attributes    707 non-null    object
 12  categories    757 non-null    object
 13  hours         606 non-null    object
dtypes: object(14)
memory usage: 88.7+ KB


Generamos un dataframe que originará un archivo unificado para tabla de BigQuery


In [9]:
df_business = pd.concat([df_holiday, df_competidores], axis=0)

Eliminamos columnas que no se utilizarán

In [10]:
df_business.drop(columns=['categories', 'attributes', 'hours'], inplace=True)

In [11]:
df_business = df_business.reset_index(drop=True)

In [15]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   901 non-null    object
 1   name          901 non-null    object
 2   address       901 non-null    object
 3   city          901 non-null    object
 4   state         901 non-null    object
 5   postal_code   901 non-null    object
 6   latitude      901 non-null    object
 7   longitude     901 non-null    object
 8   stars         901 non-null    object
 9   review_count  901 non-null    object
 10  is_open       901 non-null    object
dtypes: object(11)
memory usage: 77.6+ KB


Listamos business id generales para uso en pasos siguientes

In [12]:
yelp_business_ids = df_business['business_id'].tolist()

Listamos business id de hoteles Holiday Inn

In [11]:
holiday_business_ids = df_holiday['business_id'].tolist()

Listamos business id de hoteles de competidores

In [12]:
other_business_ids = df_competidores['business_id'].tolist()

Creamos campo is_holiday_inn para identificar los hoteles Holiday Inn

In [19]:
df_business['is_holiday_inn'] = df_business['business_id'].isin(holiday_business_ids).astype(int)

In [20]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   business_id     901 non-null    object
 1   name            901 non-null    object
 2   address         901 non-null    object
 3   city            901 non-null    object
 4   state           901 non-null    object
 5   postal_code     901 non-null    object
 6   latitude        901 non-null    object
 7   longitude       901 non-null    object
 8   stars           901 non-null    object
 9   review_count    901 non-null    object
 10  is_open         901 non-null    object
 11  is_holiday_inn  901 non-null    int64 
dtypes: int64(1), object(11)
memory usage: 84.6+ KB


Guardamos a Parquet

In [23]:
# Guardar el DataFrame como archivo Parquet
table = pa.Table.from_pandas(df_business)
pq.write_table(table, output_file_path)

# Subir el archivo resultante Parquet a la carpeta
output_blob.upload_from_filename(output_file_path)

# Limpieza del directorio temporal
os.remove(input_file_path)
os.remove(output_file_path)
os.rmdir(temp_dir)

### Checkins

Configuración de Paths de entrada y salida

In [24]:
input_file_name = "checkin.json"
output_file_name = "checkin.parquet"

# Construir la referencia al archivo de origen en la carpeta especificada
blob_name = os.path.join(input_folder_name, input_file_name)
input_blob = bucket.blob(blob_name)

# Crear directorio temporal para trabajar con los archivos
temp_dir = tempfile.mkdtemp()

# Asignar el archivo de origen al directorio temporal
input_file_path = os.path.join(temp_dir, input_file_name)
input_blob.download_to_filename(input_file_path)

# Crear referencia al archivo de salida de Parquet
output_blob_name = os.path.join(output_folder_name, output_file_name)
output_blob = bucket.blob(output_blob_name)

# Vamos a guardar los datos transformados en un nuevo archivo de Parquet
output_file_path = os.path.join(temp_dir, output_file_name)

Leemos data del json

In [25]:
data = []

with open(input_file_path, 'r') as file:
    for line in file:
        json_obj = json.loads(line)
        business_id = json_obj['business_id']
        dates_concatenated = json_obj['date']

        dates_list = [date.strip() for date in dates_concatenated.split(",")]

        for date in dates_list:
            data.append({"business_id": business_id, "date": date})

for checkin in data:
    business_id = checkin['business_id']
    date = checkin['date']

Convertimos a Dataframe

In [26]:
df_checkins = pd.DataFrame(data)

In [27]:
df_checkins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13356875 entries, 0 to 13356874
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   business_id  object
 1   date         object
dtypes: object(2)
memory usage: 203.8+ MB


Corregimos tipo de dato

In [28]:
df_checkins['date'] = pd.to_datetime(df_checkins['date'], format='%Y-%m-%d %H:%M:%S')

In [29]:
df_checkins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13356875 entries, 0 to 13356874
Data columns (total 2 columns):
 #   Column       Dtype         
---  ------       -----         
 0   business_id  object        
 1   date         datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 203.8+ MB


Aislamos los business ids que nos interesan

In [30]:
df_checkins = df_checkins[df_checkins['business_id'].isin(yelp_business_ids)]

In [31]:
df_checkins.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 101908 entries, 1503 to 13344756
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   business_id  101908 non-null  object        
 1   date         101908 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 2.3+ MB


Aislamos las fechas de nuestro alcance

In [32]:
# Rango de fechas
start_date = '2016-01-01'
end_date = '2020-12-31'

# usamos filtro booleano
filtered_df = df_checkins[(df_checkins['date'] >= start_date) & (df_checkins['date'] <= end_date)]


In [33]:
filtered_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36998 entries, 1520 to 13344756
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   business_id  36998 non-null  object        
 1   date         36998 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 867.1+ KB


In [34]:
filtered_df.reset_index(drop=True, inplace=True)

Guardamos a Parquet

In [35]:
# Guardar el DataFrame como archivo Parquet
table = pa.Table.from_pandas(filtered_df)
pq.write_table(table, output_file_path)

# Subir el archivo resultante Parquet a la carpeta
output_blob.upload_from_filename(output_file_path)

# Limpieza del directorio temporal
os.remove(input_file_path)
os.remove(output_file_path)
os.rmdir(temp_dir)

### Tips

Configuración de Paths de entrada y salida

In [13]:
input_file_name = "tip.json"
output_file_name = "tip.parquet"

# Construir la referencia al archivo de origen en la carpeta especificada
blob_name = os.path.join(input_folder_name, input_file_name)
input_blob = bucket.blob(blob_name)

# Crear directorio temporal para trabajar con los archivos
temp_dir = tempfile.mkdtemp()

# Asignar el archivo de origen al directorio temporal
input_file_path = os.path.join(temp_dir, input_file_name)
input_blob.download_to_filename(input_file_path)

# Crear referencia al archivo de salida de Parquet
output_blob_name = os.path.join(output_folder_name, output_file_name)
output_blob = bucket.blob(output_blob_name)

# Vamos a guardar los datos transformados en un nuevo archivo de Parquet
output_file_path = os.path.join(temp_dir, output_file_name)

Leemos archivo json

In [14]:
df_tips=pd.read_json(input_file_path, lines=True)

In [41]:
df_tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [15]:
df_tips = df_tips[df_tips['business_id'].isin(yelp_business_ids)]

In [18]:
# Rango de fechas
start_date = '2016-01-01'
end_date = '2020-12-31'


# usamos filtro booleano
filtered_df = df_tips[(df_tips['date'] >= start_date) & (df_tips['date'] <= end_date)]

In [45]:
df_tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4537 entries, 69 to 908328
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           4537 non-null   object        
 1   business_id       4537 non-null   object        
 2   text              4537 non-null   object        
 3   date              4537 non-null   datetime64[ns]
 4   compliment_count  4537 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 212.7+ KB


In [19]:
filtered_df.reset_index(drop=True, inplace=True)

Guardamos a Parquet

In [20]:
# Guardar el DataFrame como archivo Parquet
table = pa.Table.from_pandas(filtered_df)
pq.write_table(table, output_file_path)

# Subir el archivo resultante Parquet a la carpeta
output_blob.upload_from_filename(output_file_path)

# Limpieza del directorio temporal
os.remove(input_file_path)
os.remove(output_file_path)
os.rmdir(temp_dir)

### Reviews

Configuramos paths

In [13]:
input_file_name = "review.json"
output_file_name = "review.parquet"

# Construir la referencia al archivo de origen en la carpeta especificada
blob_name = os.path.join(input_folder_name, input_file_name)
input_blob = bucket.blob(blob_name)

# Crear directorio temporal para trabajar con los archivos
temp_dir = tempfile.mkdtemp()

# Asignar el archivo de origen al directorio temporal
input_file_path = os.path.join(temp_dir, input_file_name)
input_blob.download_to_filename(input_file_path)

# Crear referencia al archivo de salida de Parquet
output_blob_name = os.path.join(output_folder_name, output_file_name)
output_blob = bucket.blob(output_blob_name)

# Vamos a guardar los datos transformados en un nuevo archivo de Parquet
output_file_path = os.path.join(temp_dir, output_file_name)

Dividiremos el archivo en 5 partes

In [14]:
backup_folder_name = "Yelp/Backup-ETL"

In [24]:
# Número de partes en las que deseas dividir el archivo
num_parts = 5

# Calcular el tamaño aproximado deseado para cada parte
with open(input_file_path, 'r') as infile:
    total_lines = sum(1 for line in infile)
chunk_size = total_lines // num_parts

with open(input_file_path, 'r') as infile:
    chunk_count = 1
    current_chunk = []

    for line in infile:
        current_chunk.append(line)

        if len(current_chunk) >= chunk_size:
            # Crear referencia al archivo de salida de Parquet
            backup_output_blob_name = os.path.join(backup_folder_name, f'chunk_{chunk_count}.json')
            backup_output_blob = bucket.blob(backup_output_blob_name)

            # Vamos a guardar los datos transformados en un nuevo archivo de Parquet
            backup_output_file_path = os.path.join(temp_dir, f'chunk_{chunk_count}.json')

            with open(backup_output_file_path, 'w') as outfile:
                outfile.writelines(current_chunk)
            current_chunk = []
            chunk_count += 1
            # Subir el archivo resultante Parquet a la carpeta
            backup_output_blob.upload_from_filename(backup_output_file_path)

    # Guardar el último fragmento si es necesario
    if current_chunk:
        # Crear referencia al archivo de salida de Parquet
        backup_output_blob_name = os.path.join(backup_folder_name, f'chunk_{chunk_count}.json')
        backup_output_blob = bucket.blob(backup_output_blob_name)

        # Vamos a guardar los datos transformados en un nuevo archivo de Parquet
        backup_output_file_path = os.path.join(temp_dir, f'chunk_{chunk_count}.json')

        with open(backup_output_file_path, 'w') as outfile:
            outfile.writelines(current_chunk)
        # Subir el archivo resultante Parquet a la carpeta
        backup_output_blob.upload_from_filename(backup_output_file_path)

Concatenamos y guardamos en dataframe

In [15]:
# Obtener la lista de archivos en el directorio
blobs = bucket.list_blobs(prefix=backup_folder_name)

# Crear una lista para almacenar los DataFrames de los fragmentos
dataframes = []

# Cargar cada fragmento en un DataFrame y agregarlo a la lista
for blob in blobs:
    if blob.name.endswith(".json"):
        # Bajar archivos
        temp_dir_2 = tempfile.mkdtemp()
        local_file_path = os.path.join(temp_dir_2, os.path.basename(blob.name))
        blob.download_to_filename(local_file_path)

        df_fragment = pd.read_json(local_file_path, lines=True)
        dataframes.append(df_fragment)

# Borramos temporales de esta sección
os.remove(local_file_path)
os.rmdir(temp_dir_2)

# Concatenar los DataFrames en uno solo
df_reviews = pd.concat(dataframes, ignore_index=True)

In [28]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


Filtramos por los business id

In [16]:
df_reviews = df_reviews[df_reviews['business_id'].isin(yelp_business_ids)]

In [30]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50558 entries, 45 to 6990097
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   review_id    50558 non-null  object        
 1   user_id      50558 non-null  object        
 2   business_id  50558 non-null  object        
 3   stars        50558 non-null  int64         
 4   useful       50558 non-null  int64         
 5   funny        50558 non-null  int64         
 6   cool         50558 non-null  int64         
 7   text         50558 non-null  object        
 8   date         50558 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 3.9+ MB


In [17]:
# usamos filtro booleano

# Rango de fechas
start_date = '2016-01-01'
end_date = '2020-12-31'
filtered_df = df_reviews[(df_reviews['date'] >= start_date) & (df_reviews['date'] <= end_date)]

In [35]:
filtered_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27972 entries, 0 to 27971
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   review_id    27972 non-null  object        
 1   user_id      27972 non-null  object        
 2   business_id  27972 non-null  object        
 3   stars        27972 non-null  int64         
 4   useful       27972 non-null  int64         
 5   funny        27972 non-null  int64         
 6   cool         27972 non-null  int64         
 7   text         27972 non-null  object        
 8   date         27972 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 1.9+ MB


In [34]:
filtered_df.reset_index(drop=True, inplace=True)

In [40]:
# Guardar el DataFrame como archivo Parquet
table = pa.Table.from_pandas(filtered_df)
pq.write_table(table, output_file_path)

# Subir el archivo resultante Parquet a la carpeta
output_blob.upload_from_filename(output_file_path)

# Limpieza del directorio temporal
os.remove(input_file_path)
os.remove(output_file_path)